In [1]:
#importar librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model


In [4]:
# CarGa del dataSET
data_path = '.\genome.csv ' #ruta del modelo csv 
data = pd.read_csv(data_path, sep=',', quotechar='"')

# ostrar las primeras filas del dataset
print(data.head())

# Información sobre el dataset
print(data.info())
print(data.describe())


C:\Users\JERON\AppData\Local\Temp\ipykernel_21796\653600978.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_path, sep=',', quotechar='"')


         rsid chromosome  position genotype
0   rs4477212          1     72017       AA
1   rs3094315          1    742429       AA
2   rs3131972          1    742584       GG
3  rs12124819          1    766409       AA
4  rs11240777          1    788822       AG
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966977 entries, 0 to 966976
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   rsid        966977 non-null  object
 1   chromosome  966977 non-null  object
 2   position    966977 non-null  int64 
 3   genotype    966977 non-null  object
dtypes: int64(1), object(3)
memory usage: 29.5+ MB
None
           position
count  9.669770e+05
mean   7.672147e+07
std    5.640974e+07
min    3.000000e+00
25%    3.012131e+07
50%    6.667251e+07
75%    1.134776e+08
max    2.471856e+08


In [6]:
# Mapeo de genotipos a valores numéricos
def genotype_to_numeric(genotype):
    mapping = {'AA': 0, 'AG': 1, 'GG': 2}
    return mapping.get(genotype, -1)  # -1 para genotipos desconocidos

# Aplicar el mapeo a la columna 'genotype'
data['genotype_numeric'] = data['genotype'].apply(genotype_to_numeric)

# Eliminar filas con genotipos desconocidos
data = data[data['genotype_numeric'] != -1]

In [8]:
# Selección de características y etiquetas
X = data[['chromosome', 'position']].values  # Características
y = data['genotype_numeric'].values  # Etiquetas


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train)


[[3 106858436]
 [11 130582649]
 [1 94259484]
 ...
 [5 29316121]
 [5 141319706]
 [4 156566030]]


In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Ajuste y transformación de los datos de entrenamiento
X_test = scaler.transform(X_test)  # Solo transformación de los datos de prueba


ValueError: could not convert string to float: 'X'

In [ ]:
# Definición del modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Usa softmax si tienes múltiples clases
])

# Compilación del modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32)


In [ ]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Visualización de resultados
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Guardar el modelo entrenado
model.save('Projects/DNA proyect/Backend/training/dna_prediction_model.h5')
